In [ ]:
# Data2001 Group Assignment
# SID:530333591
# SA4 Region: 125-Parramatta
# SRID: choose 4326

In [ ]:
# Task1: Import all datasets

In [ ]:
# 1-1: Read all documents and check  database structure 

In [ ]:
!pip install pandas geopandas
import pandas as pd 
import geopandas as gpd

Businesses = pd.read_csv('Businesses.csv')
Income = pd.read_csv('Income.csv')


Population= pd.read_csv('Population.csv')
Stops= pd.read_csv('Stops.txt')


SA2 = gpd.read_file(
    r"/Users/kyrie/Desktop/d/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp" ).to_crs(epsg=4326)

In [ ]:
df = pd.read_csv("Businesses.csv")  
print(f"data dimension: {df.shape}")
missing = df.isna().sum()
unique = df.nunique()
dtypes = df.dtypes
samples = df.head(1).iloc[0]

In [ ]:
summary = pd.DataFrame({
 'missing_count': missing,
 'unique_count': unique,
 'dtype': dtypes,
 'sample_values': samples})
print (summary)

In [ ]:
# if primary key is composite not simple,we could check duplicate value by duplicated

In [ ]:
df = pd.read_csv("Businesses.csv")  
has_duplicates = df.duplicated(subset=['industry_code', 'sa2_code']).any()
if not has_duplicates:
    print("no duplicate found, can be a primary key.")
else:
    print("duplicate found, cannot be a primary key.")

In [ ]:
df = pd.read_csv("Income.csv")  
print(f"data dimension: {df.shape}")
missing = df.isna().sum()
unique = df.nunique()
dtypes = df.dtypes
samples = df.head(1).iloc[0]

In [ ]:
summary = pd.DataFrame({
 'missing_count': missing,
 'unique_count': unique,
 'dtype': dtypes,
 'sample_values': samples})
print (summary)

In [ ]:
df = pd.read_csv("Population.csv")  
print(f"data dimension: {df.shape}")
missing = df.isna().sum()
unique = df.nunique()
dtypes = df.dtypes
samples = df.head(1).iloc[0]

In [ ]:
summary = pd.DataFrame({
 'missing_count': missing,
 'unique_count': unique,
 'dtype': dtypes,
 'sample_values': samples})
print (summary)

In [ ]:
df = pd.read_csv("Stops.txt")  
print(f"data dimension: {df.shape}")
missing = df.isna().sum()
unique = df.nunique()
dtypes = df.dtypes
samples = df.head(1).iloc[0]

In [ ]:
summary = pd.DataFrame({
 'missing_count': missing,
 'unique_count': unique,
 'dtype': dtypes,
 'sample_values': samples})
print (summary)

In [ ]:
#filter SA2 to the ”Greater Sydney” GCC

In [ ]:

df = gpd.read_file(
    r"/Users/kyrie/Desktop/d/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp" ) 
df_syd = df[df["GCC_NAME21"] == "Greater Sydney"]
grouped = df_syd.groupby("SA4_CODE21").first().reset_index()
print(f"Greater Sydney data dimension: {df_syd.shape}")
missing = df.isna().sum()
unique = df.nunique()
dtypes = df.dtypes
samples = df.head(1).iloc[0]
print(f"Grouped data dimension: {grouped.shape}")



In [ ]:
missing = df_syd.isna().sum()
unique = df_syd.nunique()
dtypes = df_syd.dtypes
samples = df_syd.head(1).iloc[0]

summary = pd.DataFrame({
    'missing_count': missing,
    'unique_count': unique,
    'dtype': dtypes,
    'sample_values': samples
})
print(summary)


In [ ]:
#connect postgis

In [ ]:
from sqlalchemy import create_engine, inspect, text
import psycopg2
import psycopg2.extras
import json

SCHEMA = "data2001"
credentials = r"/Users/kyrie/Desktop/d/Credentials.json"

def pgconnect(credential_filepath, db_schema=SCHEMA):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)        
        host = db_conn_dict['host']        
        db_user = db_conn_dict['user']     
        db_pw = db_conn_dict['password']  
        default_db = db_conn_dict['database']  
        port = db_conn_dict['port']        

    try:
        db = create_engine(      
            f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}',
            echo=False
        )
        conn = db.connect()
        print('Connected successfully.')
    except Exception as e:
        print("Unable to connect to the database.")
        print(e)
        db, conn = None, None             
    return db, conn                       


db, conn = pgconnect(credentials, db_schema=SCHEMA)


if conn:
    conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA};"))
    conn.execute(text(f"SET search_path TO {SCHEMA};"))
    conn.execute(text("CREATE EXTENSION IF NOT EXISTS postgis;"))
    conn.commit()
    import pandas as pd
from sqlalchemy.sql import text

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(text(sqlcmd), conn, params=args)
        else:
     
            with conn.begin():
                result = conn.execute(text(sqlcmd), parameters=args).fetchall()
    
                result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered:\n", e)  
    return result 
db, conn = pgconnect(credentials)
query(conn, "SELECT PostGIS_version()")


In [ ]:
#convert data to sql

In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd


HOST     = "localhost"
PORT     = 5432
DATABASE = "postgres"
USER     = "postgres"
PASSWORD = "112233"
SCHEMA   = "data2001"

conn_str = f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
engine   = create_engine(conn_str)
conn     = engine.connect()

def add_pk(table_name: str, pk: list, conn):
    if isinstance(pk, str):
        pk = [pk]
    pk_cols = ', '.join([f'"{col}"' for col in pk])
    constraint_name = f"{table_name}_pkey"
    try:
        conn.execute(text(
            f'ALTER TABLE "{SCHEMA}"."{table_name}" '
            f'DROP CONSTRAINT IF EXISTS "{constraint_name}";'
        ))
        conn.execute(text(
            f'ALTER TABLE "{SCHEMA}"."{table_name}" '
            f'ADD CONSTRAINT "{constraint_name}" '
            f'PRIMARY KEY ({pk_cols});'
        ))
    except Exception as e:
        raise RuntimeError(f"Failed to add primary key: {e}")

def create_table(df, table_name: str, pk: list, db, schema):
    try:
        df.to_sql(
            name=table_name,
            con=db,
            schema=schema,
            if_exists="replace",
            index=False
        )
        add_pk(table_name, pk, conn)
    except Exception as e:
        raise ConnectionError(e)
    else:
        return conn.execute(
            text(f'SELECT * FROM "{schema}"."{table_name}" LIMIT 2;')
        ).fetchall()


df = pd.read_csv("businesses.csv", dtype={"industry_code": str, "sa2_code": str})
df['industry_code'] = df['industry_code'].astype(str)
df['sa2_code'] = df['sa2_code'].astype(str)


if df['industry_code'].isna().any() or df['sa2_code'].isna().any():
    raise ValueError("Need clean")
if df.duplicated(subset=['industry_code', 'sa2_code']).any():
    raise ValueError("Need clean")

pk_cols = ["industry_code", "sa2_code"]

preview = create_table(
    df         = df,
    table_name = "businesses",
    pk         = pk_cols,
    db         = engine,
    schema     = SCHEMA
)

print("Success", preview)





In [ ]:
res = conn.execute(text('SELECT COUNT(*) FROM "data2001"."businesses";')).fetchone()
print("Row number ", res)

In [ ]:

df['sa2_code'] = df['sa2_code'].astype(str)
business_df = pd.read_csv('businesses.csv')
business_df['sa2_code'] = business_df['sa2_code'].astype(str)
stops_df = pd.read_csv('stops.txt', sep='\t')
stops_df['sa2_code'] = stops_df['sa2_code'].astype(str)
poi_df = pd.read_csv('poi_greater_sydney.csv')
poi_df['sa2_code'] = poi_df['sa2_code'].astype(str)


poi_count = poi_df.groupby('sa2_code').size().reset_index(name='poi_count')


df = df.merge(business_df[['sa2_code', 'total_businesses']], on='sa2_code', how='left')
df = df.merge(stops_df[['sa2_code', 'stop_count']], on='sa2_code', how='left')
df = df.merge(poi_count, on='sa2_code', how='left')


In [ ]:
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine, text

shapefile_path = r"/Users/kyrie/Desktop/d/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp"
df = gpd.read_file(shapefile_path)
df_syd = df[df["GCC_NAME21"] == "Greater Sydney"].copy()  


HOST     = "localhost"
PORT     = 5432
DATABASE = "postgres"
USER     = "postgres"
PASSWORD = "112233"
SCHEMA   = "data2001"

conn_str = f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
engine   = create_engine(conn_str)
conn     = engine.connect()

def add_pk(table_name: str, pk: list, conn):
    if isinstance(pk, str):
        pk = [pk]
    pk_cols = ', '.join([f'"{col}"' for col in pk])
    constraint_name = f"{table_name}_pkey"
    try:
        conn.execute(text(
            f'ALTER TABLE "{SCHEMA}"."{table_name}" '
            f'DROP CONSTRAINT IF EXISTS "{constraint_name}";'
        ))
        conn.execute(text(
            f'ALTER TABLE "{SCHEMA}"."{table_name}" '
            f'ADD CONSTRAINT "{constraint_name}" '
            f'PRIMARY KEY ({pk_cols});'
        ))
    except Exception as e:
        raise RuntimeError(f"Failed to add primary key: {e}")

def create_table(df, table_name: str, pk: list, db, schema):
    try:
        df.to_sql(
            name=table_name,
            con=db,
            schema=schema,
            if_exists="replace",
            index=False
        )
        add_pk(table_name, pk, conn)
    except Exception as e:
        raise ConnectionError(e)
    else:
        return conn.execute(
            text(f'SELECT * FROM "{schema}"."{table_name}" LIMIT 2;')
        ).fetchall()

table_name = "sa2_greater_sydney"
pk_cols = ["SA2_CODE21"]

df_syd["geometry"] = df_syd["geometry"].apply(lambda x: x.wkt)

preview = create_table(
    df=df_syd,
    table_name=table_name,
    pk=pk_cols,
    db=engine,
    schema=SCHEMA
)

print("Success ", preview)


In [ ]:
print(engine)


In [ ]:

import pandas as pd
df_stops = pd.read_csv("Stops.txt")

table_name = "stops"
pk_cols = ["stop_id"]

preview = create_table(
    df=df_stops,
    table_name=table_name,
    pk=pk_cols,
    db=engine,
    schema=SCHEMA
)

print("success", preview)


In [ ]:
import geopandas as gpd

# secondary
secondary_gdf = gpd.read_file('/Users/kyrie/Desktop/d/catchments/catchments_secondary.shp')
if hasattr(secondary_gdf["geometry"].iloc[0], "wkt"):
    secondary_gdf["geometry"] = secondary_gdf["geometry"].apply(lambda x: x.wkt)
create_table(
    df=secondary_gdf,
    table_name="school_catchments_secondary",
    pk=["USE_ID"],
    db=engine,
    schema=SCHEMA
)

# future
future_gdf = gpd.read_file('/Users/kyrie/Desktop/d/catchments/catchments_future.shp')
if hasattr(future_gdf["geometry"].iloc[0], "wkt"):
    future_gdf["geometry"] = future_gdf["geometry"].apply(lambda x: x.wkt)
create_table(
    df=future_gdf,
    table_name="school_catchments_future",
    pk=["USE_ID"],
    db=engine,
    schema=SCHEMA
)

# primary
primary_gdf = gpd.read_file('/Users/kyrie/Desktop/d/catchments/catchments_primary.shp')
if hasattr(primary_gdf["geometry"].iloc[0], "wkt"):
    primary_gdf["geometry"] = primary_gdf["geometry"].apply(lambda x: x.wkt)
create_table(
    df=primary_gdf,
    table_name="school_catchments_primary",
    pk=["USE_ID"],
    db=engine,
    schema=SCHEMA
)


In [ ]:
import pandas as pd


df_pop = pd.read_csv("Population.csv")

table_name = "population"
pk_cols = ["sa2_code"]

preview = create_table(
    df=df_pop,
    table_name=table_name,
    pk=pk_cols,
    db=engine,
    schema=SCHEMA
)

print("Success", preview)


In [ ]:
import pandas as pd

df_income = pd.read_csv("Income.csv")

table_name = "income"
pk_cols = ["sa2_code21"]

preview = create_table(
    df=df_income,
    table_name=table_name,
    pk=pk_cols,
    db=engine,
    schema=SCHEMA
)

print("Success", preview)


In [ ]:
#Task 2 NSW Points of interest API

In [ ]:
import requests

def get_poi_bbox(minx, miny, maxx, maxy, srid=4326, max_records=1000):
    url = "https://maps.six.nsw.gov.au/arcgis/rest/services/public/NSW_POI/MapServer/0/query"
    params = {
        "f": "geojson",
        "geometry": f"{minx},{miny},{maxx},{maxy}",
        "geometryType": "esriGeometryEnvelope",
        "inSR": srid,
        "spatialRel": "esriSpatialRelIntersects",
        "outFields": "*",
        "returnGeometry": "true",
        "resultRecordCount": max_records,
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None


In [ ]:
import geopandas as gpd
import pandas as pd
import requests
import time
import os

shapefile_path = r"/Users/kyrie/Desktop/d/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp"
df = gpd.read_file(shapefile_path)
df_syd = df[df["GCC_NAME21"] == "Greater Sydney"].copy().reset_index(drop=True)


if os.path.exists("poi_greater_sydney.csv"):
    df_poi_done = pd.read_csv("poi_greater_sydney.csv")
    done_sa2 = set(df_poi_done["sa2_code"].unique())
    print(f"Fininshed {len(done_sa2)}  SA2")
else:
    df_poi_done = pd.DataFrame()
    done_sa2 = set()
    print("still not finished")


df_syd_left = df_syd[~df_syd["SA2_CODE21"].isin(done_sa2)].reset_index(drop=True)
print(f"Remain to be get：{len(df_syd_left)}")


def get_poi_bbox(minx, miny, maxx, maxy, srid=4326, max_records=1000):
    url = "https://maps.six.nsw.gov.au/arcgis/rest/services/public/NSW_POI/MapServer/0/query"
    params = {
        "f": "geojson",
        "geometry": f"{minx},{miny},{maxx},{maxy}",
        "geometryType": "esriGeometryEnvelope",
        "inSR": srid,
        "spatialRel": "esriSpatialRelIntersects",
        "outFields": "*",
        "returnGeometry": "true",
        "resultRecordCount": max_records,
    }
    response = requests.get(url, params=params, timeout=10)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

poi_list = []

for idx, row in df_syd_left.iterrows():
    minx, miny, maxx, maxy = row.geometry.bounds
    sa2_code = row['SA2_CODE21']
    try:
        geojson = get_poi_bbox(minx, miny, maxx, maxy)
    except Exception as e:
        print(f"SA2 {sa2_code} fail: {e}")
        continue
    if geojson and 'features' in geojson:
        features = geojson['features']
        for feature in features:
            props = feature['properties']
            props['sa2_code'] = sa2_code
            poi_list.append(props)
    print(f"Finished {idx+1}{len(df_syd_left)} SA2（Now：{len(done_sa2)+idx+1}/{len(df_syd)}）")
    time.sleep(1)
 
    if (idx+1) % 50 == 0:
        pd.DataFrame(poi_list).to_csv("poi_new_partial.csv", index=False)
        print(f"Have finished into poi_new_partial.csv")


df_new = pd.DataFrame(poi_list)
if not df_poi_done.empty:
    df_all = pd.concat([df_poi_done, df_new], ignore_index=True)
else:
    df_all = df_new

df_all.to_csv("poi_greater_sydney.csv", index=False)
print(f"all done，total{df_all['sa2_code'].nunique()}SA2，{df_all.shape[0]}POI")



In [ ]:
#Check for the result

In [ ]:
import pandas as pd

df_poi = pd.read_csv("poi_greater_sydney.csv")
print(df_poi.shape)
print(df_poi.head())
print(df_poi['sa2_code'].nunique()) 


In [ ]:
print(df_poi.columns)
print(df_poi.head())


In [ ]:
preview = create_table(
    df=df_poi,
    table_name="nsw_poi_sa2",
    pk=["objectid", "sa2_code"], 
    db=engine,
    schema=SCHEMA
)


In [ ]:
#task3 Compute score for well-resourced

In [ ]:
sa2_gdf = gpd.read_file( r"/Users/kyrie/Desktop/d/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp").to_crs(epsg=4326)


In [ ]:
import geopandas as gpd


primary_gdf = gpd.read_file('/Users/kyrie/Desktop/d/catchments/catchments_primary.shp').to_crs(epsg=4326)
secondary_gdf = gpd.read_file('/Users/kyrie/Desktop/d/catchments/catchments_secondary.shp').to_crs(epsg=4326)
future_gdf = gpd.read_file('/Users/kyrie/Desktop/d/catchments/catchments_future.shp').to_crs(epsg=4326)


sa2_gdf = gpd.read_file(r"/Users/kyrie/Desktop/d/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp").to_crs(epsg=4326)


In [ ]:

primary_join = gpd.sjoin(primary_gdf, sa2_gdf[['SA2_CODE21', 'geometry']], how='left', predicate='intersects')
secondary_join = gpd.sjoin(secondary_gdf, sa2_gdf[['SA2_CODE21', 'geometry']], how='left', predicate='intersects')
future_join = gpd.sjoin(future_gdf, sa2_gdf[['SA2_CODE21', 'geometry']], how='left', predicate='intersects')


primary_count = primary_join.groupby('SA2_CODE21').size().reset_index(name='school_count_primary')
secondary_count = secondary_join.groupby('SA2_CODE21').size().reset_index(name='school_count_secondary')
future_count = future_join.groupby('SA2_CODE21').size().reset_index(name='school_count_future')


In [ ]:
df['school_catchment_total'] = (
    df['school_count_primary'].fillna(0) +
    df['school_count_secondary'].fillna(0) +
    df['school_count_future'].fillna(0)
)


In [ ]:

df = pd.read_csv('population.csv')
df['sa2_code'] = df['sa2_code'].astype(str)
primary_count['SA2_CODE21'] = primary_count['SA2_CODE21'].astype(str)
secondary_count['SA2_CODE21'] = secondary_count['SA2_CODE21'].astype(str)
future_count['SA2_CODE21'] = future_count['SA2_CODE21'].astype(str)


df = df.merge(primary_count, left_on='sa2_code', right_on='SA2_CODE21', how='left')
df = df.merge(secondary_count, left_on='sa2_code', right_on='SA2_CODE21', how='left')
df = df.merge(future_count, left_on='sa2_code', right_on='SA2_CODE21', how='left')


In [ ]:
df['school_catchment_total'] = (
    df['school_count_primary'].fillna(0) +
    df['school_count_secondary'].fillna(0) +
    df['school_count_future'].fillna(0)
)


In [ ]:
print(stops_df.columns)
stops_df.head()



In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


stops_df = pd.read_csv('stops.txt')

stops_gdf = gpd.GeoDataFrame(
    stops_df,
    geometry=stops_df.apply(lambda row: Point(float(row['stop_lon']), float(row['stop_lat'])), axis=1),
    crs='EPSG:4326'
)


In [ ]:
sa2_gdf = gpd.read_file(r"/Users/kyrie/Desktop/d/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp").to_crs('EPSG:4326')


In [ ]:
stops_with_sa2 = gpd.sjoin(stops_gdf, sa2_gdf[['SA2_CODE21', 'geometry']], how='left', predicate='within')


In [ ]:
stops_count = stops_with_sa2.groupby('SA2_CODE21').size().reset_index(name='stop_count')
stops_count['SA2_CODE21'] = stops_count['SA2_CODE21'].astype(str)


In [ ]:
df['sa2_code'] = df['sa2_code'].astype(str)
stops_count['SA2_CODE21'] = stops_count['SA2_CODE21'].astype(str)

if 'SA2_CODE21' in df.columns:
    df = df.drop(columns=['SA2_CODE21'])

df = df.merge(stops_count, left_on='sa2_code', right_on='SA2_CODE21', how='left')
df = df.drop(columns=['SA2_CODE21'])


In [ ]:
# Check before merge

In [ ]:

business_sa2 = business_df.groupby('sa2_code')['total_businesses'].sum().reset_index()

df['business_per_1000'] = df['total_businesses'] / df['population'] * 1000
df['schools_per_1000_young'] = df['school_catchment_total'] / df['population_0_19'] * 1000


In [ ]:
df = df[df['population'] >= 100]


In [ ]:
df = df.merge(business_sa2, on='sa2_code', how='left')


In [ ]:
print(df.columns)


In [ ]:
import pandas as pd


poi_df = pd.read_csv('poi_greater_sydney.csv')
poi_df['sa2_code'] = poi_df['sa2_code'].astype(str)


poi_count = poi_df.groupby('sa2_code').size().reset_index(name='poi_count')


In [ ]:
df['sa2_code'] = df['sa2_code'].astype(str)
df = df.merge(poi_count, on='sa2_code', how='left')


In [ ]:
from scipy.special import expit


for col in ['business_per_1000', 'stop_count', 'schools_per_1000_young', 'poi_count']:
    df[f'z_{col}'] = (df[col] - df[col].mean()) / df[col].std()


In [ ]:
df['score_raw'] = (
    df['z_business_per_1000']
    + df['z_stop_count']
    + df['z_schools_per_1000_young']
    + df['z_poi_count']
)

df['score'] = expit(df['score_raw'])


In [ ]:
result = df[['sa2_code', 'score']].sort_values(by='score', ascending=False)
print(result.head(100))  
result.to_csv('sa2_well_resourced_score.csv', index=False)


In [ ]:
#select only sa2_code start with 125

In [ ]:

result_125_full = result_full[result_full['sa2_code'].str.startswith('125')]

print(result_125_full)


result_125_full.to_csv('sa2_125area_well_resourced_score.csv', index=False)



In [ ]:
#task4 Summarise and Visualise and Scrutinise

In [ ]:

result_125_full = result_full[result_full['sa2_code'].str.startswith('125')].copy()


In [ ]:
# Summary of Main Findings
# Highest and Lowest Scoring SA2 Regions
# Top-scoring SA2s in the 125 (Parramatta) region include Ermington – Rydalmere, Granville – Clyde, Guildford West – Merrylands West, and North Parramatta. These areas are distinctly well-resourced compared to the regional average.
# Lowest-scoring SA2s are Parramatta – South, Berala, Auburn – North, and Auburn – South, where access to various facilities and services is comparatively limited

In [ ]:
#Score Distribution and Spatial Patterns
#The bar chart reveals a pronounced disparity in scores across the 125 region SA2s, with a handful of areas significantly outperforming the rest.
#The choropleth map clearly shows that the highest scores cluster around the core of Parramatta and key transport nodes. This includes areas near the Parramatta CBD, major railway lines, and riverside precincts.
#Lower scores are typically found in peripheral or traditionally industrial SA2s, or older residential neighborhoods with limited recent investment in amenities.

#Urban and Locational Context
#Parramatta functions as Sydney’s “second CBD,” a major economic, educational, and transport hub for Western Sydney. SA2s adjacent to transport interchanges, commercial precincts, and newly developed mixed-use neighborhoods tend to score the highest.
#Many high-scoring regions have benefitted from recent infrastructure upgrades and urban renewal, whereas lower-scoring regions may be lagging due to legacy zoning (e.g., industrial uses) or limited public transport access.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.bar(result_125_full['sa2_name'], result_125_full['score'], color='#4682b4')
plt.ylabel('Well-Resourced Score')
plt.title('Score per SA2 Region (SA2 Code starts with 125)')
plt.xticks(rotation=60, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns

corr_cols = ['z_business', 'z_stops', 'z_schools', 'z_poi', 'score']
corr_matrix_125 = result_125_full[corr_cols].corr()

plt.figure(figsize=(7, 5))
sns.heatmap(corr_matrix_125, annot=True, cmap='rocket', vmin=-1, vmax=1)
plt.title('Correlation Matrix (SA2 Code starts with 125)')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


df_income = pd.read_csv("Income.csv")  # Income.csv 必须含 sa2_code, median_income


result_125_full['sa2_code'] = result_125_full['sa2_code'].astype(str)
df_income['sa2_code'] = df_income['sa2_code'].astype(str)
result_125_income = result_125_full.merge(
    df_income[['sa2_code', 'median_income']], on='sa2_code', how='left'
)


plt.figure(figsize=(7, 6))
plt.scatter(result_125_income['score'], result_125_income['median_income'], alpha=0.75, color='#4682b4')
plt.xlabel('Well-Resourced Score')
plt.ylabel('Median Income ($)')
plt.title('Parramatta (125 region): Score vs Median Income')
plt.grid(True)
plt.tight_layout()
plt.show()

corr = result_125_income['score'].corr(result_125_income['median_income'])
print(f'Correlation coefficient: {corr:.2f}')


In [ ]:
import geopandas as gpd


sa2_gdf['SA2_CODE21'] = sa2_gdf['SA2_CODE21'].astype(str)
plot_gdf_125 = sa2_gdf[sa2_gdf['SA2_CODE21'].str.startswith('125')].merge(
    result_125_full[['sa2_code', 'score']],
    left_on='SA2_CODE21', right_on='sa2_code', how='left'
)

fig, ax = plt.subplots(1, 1, figsize=(10, 7))
plot_gdf_125.plot(column='score', ax=ax, cmap='OrRd', legend=True, edgecolor='black')
ax.set_title('Well-Resourced Score (SA2 Code starts with 125)')
ax.set_axis_off()
plt.show()


In [ ]:
#Bar Chart
#Clearly illustrates a “long-tail” distribution: a small number of SA2s have outstanding resource access, while many others trail behind.
#High scores tend to coincide with a balance of strong transport, business, education, and POI (points of interest) metrics.

#Correlation Matrix
#The heatmap shows that the number of public transport stops (z_stops) and POI count (z_poi) are the strongest drivers of the overall resource score.
#The business and school metrics also contribute, but their correlation with the final score is weaker in some SA2s—likely reflecting local land use patterns (e.g., residential vs. commercial zones).

#Example:
#In Ermington – Rydalmere and Granville – Clyde, high scores are driven by a strong presence of transport stops and a high density of POIs. In contrast, some areas such as Berala may have a relatively high school count but lack business and POI density, resulting in a lower overall score.

In [ ]:
#3. Drivers of Regional Differences
#Transport Stops and POI Density:
#These two factors are most influential in determining high scores. For example, SA2s intersected by train lines or close to the Parramatta CBD typically have both high transport stop density and diverse POI access, driving up their scores.

#Schools and Businesses:
#While important, their effect is less pronounced in some SA2s. In some lower-scoring regions, even a reasonable number of schools may not compensate for the lack of transport and general amenities.

#Concrete Example:

#Auburn – North may have several schools (high z_schools), but its relatively low business density and POI diversity—combined with fewer transport connections—lead to a much lower total resource score.
#Conversely, Guildford West – Merrylands West scores high not only for schools but also for an abundance of public transport and POIs, illustrating how these components interact to determine overall outcomes.